In [ ]:
from langchain_experimental.tools import PythonREPLTool

ModuleNotFoundError: No module named 'langchain_experimental'

In [1]:
from langchain_experimental.tools import PythonREPLTool
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent, Tool
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
import pandas as pd
import os
#load variable from .env
load_dotenv()  
openai_api_key = os.getenv('OPENAI_API_KEY')

In [11]:
# Define the data as a list of lists
data = [
    [10001, '1953-09-02', 'Georgi', 'Facello', 'M', '1986-06-26'],
    [10002, '1964-06-02', 'Bezalel', 'Simmel', 'F', '1985-11-21'],
    [10003, '1959-12-03', 'Parto', 'Bamford', 'M', '1986-08-28'],
    [10004, '1954-05-01', 'Chirstian', 'Koblick', 'M', '1986-12-01'],
    [10005, '1955-01-21', 'Kyoichi', 'Maliniak', 'M', '1989-09-12'],
    [10006, '1953-04-20', 'Anneke', 'Preusig', 'F', '1989-06-02'],
    [10007, '1957-05-23', 'Tzvetan', 'Zielinski', 'F', '1989-02-10'],
    [10008, '1958-02-19', 'Saniya', 'Kalloufi', 'M', '1994-09-15'],
    [10009, '1952-04-19', 'Sumant', 'Peac', 'F', '1985-02-18'],
    [10010, '1963-06-01', 'Duangkaew', 'Piveteau', 'F', '1989-08-24']
]

# Define the column names
columns = ['emp_no', 'birth_date', 'first_name', 'last_name', 'gender', 'hire_date']

# Create the DataFrame
df = pd.DataFrame(data, columns=columns)
dataframe = df

In [14]:
def main():
    print("Start agent execution...")
    python_agent_executor = create_python_agent(
        llm=ChatOpenAI(temperature=0, model="gpt-4", api_key = openai_api_key),
        tool=PythonREPLTool(),
        agent_type= AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
    )
    
    csv_agent = create_csv_agent(
        llm=ChatOpenAI(temperature=0, model="gpt-4", api_key = openai_api_key),
        path = r"C:\Users\(Ai)AiSukmoren\Desktop\langchain_demo\doc\episode_info.csv",
        agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
    )
    
    pandas_agent = create_pandas_dataframe_agent(
        llm=ChatOpenAI(temperature=0, model="gpt-4", api_key = openai_api_key),
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        df = dataframe,
        verbose=True,
        handle_parsing_errors=True,
    )
    
    grand_agent = initialize_agent(
        tools=[
            Tool(
                name="PythonAgent",
                func=python_agent_executor.run,
                description="""useful when you need to transform natural language and write from it python and execute the python code,
                              returning the results of the code execution,
                            DO NOT SEND PYTHON CODE TO THIS TOOL""",
            ),
            Tool(
                name="CSVAgent",
                func=csv_agent.run,
                description="""useful when you need to answer question over episode_info.csv fileepisode_info.csv file,
                             takes an input the entire question and returns the answer after running pandas calculations""",
            ),
            Tool(
                name="PandasAgent",
                func=pandas_agent.run,
                description="""useful when you need to answer question over pandas dataframe that was provide by the name df,
                             takes an input the entire question and returns the answer after running pandas calculations""",
            ),
        ],
        llm=ChatOpenAI(temperature=0, model="gpt-4"),
        agent_type=AgentType.OPENAI_FUNCTIONS,
        verbose=True,
        handle_parsing_errors=True,
    )
    
    grand_agent.run(
     """from the data frame provide which gender has the highest number?
     """)

if __name__ == "__main__":
    main()

Start agent execution...


> Entering new AgentExecutor chain...
I need to calculate the count of each gender in the provided dataframe.
Action: PandasAgent
Action Input: from the data frame provide which gender has the highest number?

> Entering new AgentExecutor chain...
Thought: To find out which gender has the highest number, I need to count the number of occurrences of each gender in the 'gender' column.
Action: python_repl_ast
Action Input: df['gender'].value_counts()
Observation: gender
M    5
F    5
Name: count, dtype: int64
Thought:The count of both genders 'M' and 'F' is the same in the dataframe. 
Final Answer: Both genders 'M' and 'F' have the same count.

> Finished chain.

Observation: Both genders 'M' and 'F' have the same count.
Thought:I now know the final answer
Final Answer: Both genders 'M' and 'F' have the same count.

> Finished chain.
